In [1]:
import pandas as pd
from time import time
import re
import string
import joblib

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [3]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from cleantext import clean
import pyLDAvis
import matplotlib.pyplot as plt
import pyLDAvis.sklearn

In [4]:
RANDOM_SEED=123
N_FEATURES = 1000

In [5]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'same', "hadn't", 'doesn', 'again', 'between', 'the', 'more', 'hers', 'to', 'my', 'both', 'has', "you're", 'or', 're', 'needn', 'over', 'that', 'have', 'because', 'won', 'here', 'd', 'doing', 's', 'don', 'i', 'theirs', 'he', 'be', 'nor', 'each', 'where', 'not', 'ourselves', 'what', "you'd", 'been', 'who', 'll', 'yourself', 'should', "don't", "should've", 'myself', "shan't", 'for', "couldn't", 'but', 'above', 'as', 'shan', 'when', 'by', 'after', 'were', "it's", 'his', 'isn', 'o', "doesn't", 'ma', 'me', 'why', 'of', 'they', 'it', "she's", 'about', "shouldn't", 'up', 'and', 'an', "wouldn't", 'she', 'did', 'weren', "won't", 'will', 'most', 'further', 'only', 'at', 'themselves', 'you', "that'll", 'into', 'too', "weren't", 'out', 'aren', 'with', 'if', 'these', 'herself', 'now', "you've", 'few', 'other', 'their', 'in', 'its', 'there', 'mustn', 'while', 'below', 'itself', 'such', 'yours', 'couldn', 'are', 'himself', 'being', 'all', 'am', 'having', 'which', 'during', 'y', 'own', 'we', 'is', 't

## Очистка текстов

In [6]:
def tolower(s):
    return s.lower()

def remove_code_blocks(s):
    return re.sub(r"```[^\S\r\n]*[a-z]*\n.*?\n```", '', s, 0, re.DOTALL)

def remove_long_words(s, good_len=15):
    return " ".join([word for word in s.split() if len(word) < good_len])

def remove_stopw(s):
    return " ".join([word for word in s.split() if word not in stop_words])

def remove_custom_stopw(s):
    stopw = ["line", "file", "jetbrains", "idea", "pycharm", "type", "import", "class"]
    stopw += ["type", "int", "float", "python", "run", "traceback", "version", ""]
    
    return " ".join([word for word in s.split() if word not in stopw])

def remove_unicode(s):
    text_encode = s.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

def remove_stuff(s):
    s = re.sub(r"@\S+", "", s)
    s = re.sub(r"\$", "", s)
    s = re.sub(r"https?:\/\/.*[\r\n]*", "", s)
    s = re.sub(r"#", "", s)
    punct = set(string.punctuation) 
    s = "".join([ch for ch in s if ch not in punct])
    return s

def stem_lem(s):
    stemmer = PorterStemmer()
    s = " ".join([stemmer.stem(word) for word in s.split()])
    return s

def remove_java_packages(s):
     return " ".join([word for word in s.split() if not len(word.split('.')) > 2])
    
def clear_str(s):
    s = remove_code_blocks(s)
    s = tolower(s)
    s = remove_java_packages(s)
    s = remove_stopw(s)
    s = remove_custom_stopw(s)
    s = remove_unicode(s)
    s = remove_stuff(s)
    s = stem_lem(s)
    s = remove_long_words(s)
    return s

In [7]:
data = pd.read_json('pycharm_issues.json', lines=True)

In [8]:
def clean_data_descr(s):
    if not s:
        return ""
    s = clear_str(s)
    
    s = clean(f"{s}",
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    no_emoji=True,
    
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="",
    lang="en"                       # set to 'de' for German special handling
)
    return s

In [9]:
cleaning_lambda = lambda d: clean_data_descr(d)

In [10]:
t0 = time()

data["str_data"] = data["summary"].astype(str) + data["description"].astype(str)
data["str_data"] = data["str_data"].apply(cleaning_lambda)


time() - t0

25.19657301902771

### Получим данные про версии 2020.2 и 2020.3

In [11]:
v_2020_2 = data['Affected versions'].apply(pd.Series).eq('2020.2').any(1)
data_v2020_2_str = data[v_2020_2]["str_data"]

v_2020_3 = data['Affected versions'].apply(pd.Series).eq('2020.3').any(1)
data_v2020_3_str = data[v_2020_3]["str_data"]

/var/folders/l1/6p4_ds8168q6gcpff7fyvj_c0000gn/T/ipykernel_40167/3316689327.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  v_2020_2 = data['Affected versions'].apply(pd.Series).eq('2020.2').any(1)
/var/folders/l1/6p4_ds8168q6gcpff7fyvj_c0000gn/T/ipykernel_40167/3316689327.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  v_2020_3 = data['Affected versions'].apply(pd.Series).eq('2020.3').any(1)


In [12]:
t0 = time()

data["str_data"] = data["summary"].astype(str) + data["description"].astype(str)
data_docs_cleaned = data["str_data"].apply(cleaning_lambda)


time() - t0

29.442588806152344

In [13]:
data_samples = list(data_docs_cleaned)

### Векторизуем текст

In [14]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=N_FEATURES, stop_words="english"
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 0.923s.



In [23]:
def grid_search_lda(search_params, tf):
    lda = LatentDirichletAllocation(random_state=RANDOM_SEED)
    model = GridSearchCV(lda, param_grid=search_params, verbose=10)
    model.fit(tf)
    
    best_lda_model = model.best_estimator_

    print("Best Model's Params: ", model.best_params_)

    print("Best Log Likelihood Score: ", model.best_score_)

    print("Model Perplexity: ", best_lda_model.perplexity(tf))
    
    joblib.dump(best_lda_model, 'best_lda.pkl')
    return best_lda_model

In [16]:
def visualise_topics(data, model, tf_vectorizer, name):
    data_tf = tf_vectorizer.transform(data)
    pyLDAvis.enable_notebook()
    visualization = pyLDAvis.sklearn.prepare(model, data_tf, tf_vectorizer)
    pyLDAvis.save_html(visualization, name)
    return visualization

In [17]:
# search_params = {'n_components': [5, 6, 7, 8, 9, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
search_params = {'n_components': [5, 8, 10, 12, 15, 20]}
model = grid_search_lda(search_params)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START n_components=5..............................................
[CV 1/5; 1/6] END .........n_components=5;, score=-850971.094 total time= 1.0min
[CV 2/5; 1/6] START n_components=5..............................................
[CV 2/5; 1/6] END .........n_components=5;, score=-892233.097 total time= 1.1min
[CV 3/5; 1/6] START n_components=5..............................................
[CV 3/5; 1/6] END .........n_components=5;, score=-887515.008 total time= 1.1min
[CV 4/5; 1/6] START n_components=5..............................................
[CV 4/5; 1/6] END .........n_components=5;, score=-928375.728 total time= 1.0min
[CV 5/5; 1/6] START n_components=5..............................................
[CV 5/5; 1/6] END .........n_components=5;, score=-904276.174 total time=  59.7s
[CV 1/5; 2/6] START n_components=8..............................................
[CV 1/5; 2/6] END .........n_components=8;, score

In [18]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.093027 -0.092063       1        1  11.668017
0      0.105765  0.083872       2        1  11.536576
5      0.077302  0.256683       3        1   8.547360
13     0.025111 -0.024175       4        1   8.528811
8      0.086045  0.036339       5        1   8.510701
9      0.056754 -0.054247       6        1   8.283914
10     0.146050  0.186567       7        1   7.839670
12    -0.166903 -0.016484       8        1   6.355073
3      0.060594  0.087281       9        1   5.967691
11     0.050956 -0.174956      10        1   5.459461
2      0.003846 -0.190848      11        1   4.042079
14    -0.234320  0.063684      12        1   3.928560
6      0.095158 -0.103204      13        1   3.731978
4      0.006388 -0.107247      14        1   3.072989
7     -0.405773  0.048796      15        1   2.527118, topic_info=       Term        Freq       Total Category  logprob  loglift
599      os  165.000000  165.000000  Default  30.0000  30.0000
848    step  245.000000  245.000000  Default  29.0000  29.0000
753  result  283.000000  283.000000  Default  28.0000  28.0000
273   error  189.000000  189.000000  Default  27.0000  27.0000
775  screen  128.000000  128.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
775  screen   53.755664  128.290885  Topic15  -2.3400   2.8082
458     jre   50.957026  140.506682  Topic15  -2.3934   2.6638
508    long    6.991193   25.972898  Topic15  -4.3798   2.3657
986    work    5.437031  220.370341  Topic15  -4.6312  -0.0240
365   handl    2.340925   11.135780  Topic15  -5.4739   2.1185

[754 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3         1  0.706157   absolut
4         2  1.049664  abstract
10        1  0.312923     activ
10        3  0.028448     activ
10        4  0.056895     activ
...     ...       ...       ...
992      10  0.049486       wsl
993      15  0.906923      xfff
994      15  1.133158   xffffff
996      15  1.007182        xx
997       4  0.658910      yaml

[2184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 6, 14, 9, 10, 11, 13, 4, 12, 3, 15, 7, 5, 8])

In [19]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.105765 -0.083872       1        1  14.768498
1     -0.093027  0.092063       2        1  11.861488
5     -0.077302 -0.256683       3        1   9.070867
13    -0.025111  0.024175       4        1   8.617395
8     -0.086045 -0.036339       5        1   8.447298
10    -0.146050 -0.186567       6        1   7.957502
3     -0.060594 -0.087281       7        1   6.851534
9     -0.056754  0.054247       8        1   6.149099
11    -0.050956  0.174956       9        1   5.347120
6     -0.095158  0.103204      10        1   4.979776
12     0.166903  0.016484      11        1   4.161806
14     0.234320 -0.063684      12        1   3.799206
2     -0.003846  0.190848      13        1   3.179202
7      0.405773 -0.048796      14        1   2.615045
4     -0.006388  0.107247      15        1   2.194164, topic_info=        Term        Freq       Total Category  logprob  loglift
599       os  130.000000  130.000000  Default  30.0000  30.0000
885     test  114.000000  114.000000  Default  29.0000  29.0000
848     step  201.000000  201.000000  Default  28.0000  28.0000
753   result  235.000000  235.000000  Default  27.0000  27.0000
273    error  135.000000  135.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
866  support    3.486730   43.531392  Topic15  -4.7090   1.2949
761      run    4.857119   95.955391  Topic15  -4.3775   0.8359
976     warn    3.978902  118.725934  Topic15  -4.5769   0.4236
591     open    2.996588  117.501763  Topic15  -4.8605   0.1504
595   option    2.483903   57.116818  Topic15  -5.0481   0.6841

[752 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         5  0.676493      abil
0        15  0.169123      abil
3         2  0.852846   absolut
4         1  1.038848  abstract
10        2  0.363449     activ
...     ...       ...       ...
992       6  0.183698       wsl
993      14  1.098304      xfff
994      14  0.915126   xffffff
996      14  0.997565        xx
997       4  0.796699      yaml

[2017 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 14, 9, 11, 4, 10, 12, 7, 13, 15, 3, 8, 5])

## Вывод

Если не фильтровать текст кода, то токены из обрывков кода начинают заполнять все топики, поэтому приходится его полностью удалить.  

При обучении на количестве топиков от 5 до 20, лучший результат обучения по perplexity получился на 15 топиках.  
Однако, если посмотреть на визуализацию топиков для 2 разных релизов, то, во-первых, разделение на топики практически одинаковое, а, во-вторых, топики сложно интерпретировать и отделить друг от друга.  
Хотя, можно сделать вывод, что 12-я тема из второй визуализации ответственна за UI, тогда как темы в группе слева состоят из каких-то самых популярных слов и терминов для любого проекта.  
14-я тема фокусируется на проблемах, связанных с операционной системой и библиотеками. Она также присутствует в обоих релизах.  
11-я тема -- про виртуализацию и сборку. Также сохраняется.  
13-я тема -- различные ошибки.  
9-я тема -- docker, remote connection, ssh, web, networks.  
15-я -- jupyter notebooks, kernels, sessions etc.  
8-я -- дебаггер, breakpoints, console, terminal.  
7-я -- UI.  
1 -- непосредственно тело кода и работа с ним в PyCharm.  
Для остальных тем труднее выделить их смысл.   
Топики абсолютно не поменялись в новом релизе.


Попробуем все таки посчитать для меньшего количества топиков

In [53]:
N_FEATURES = 2000

In [54]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=N_FEATURES, stop_words="english"
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 1.050s.



In [55]:
search_params = {'n_components': [3, 4, 5, 6, 7, 8]}
model = grid_search_lda(search_params, tf)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START n_components=3..............................................
[CV 1/5; 1/6] END .........n_components=3;, score=-983530.135 total time= 1.3min
[CV 2/5; 1/6] START n_components=3..............................................
[CV 2/5; 1/6] END ........n_components=3;, score=-1019405.475 total time= 1.2min
[CV 3/5; 1/6] START n_components=3..............................................
[CV 3/5; 1/6] END ........n_components=3;, score=-1016676.113 total time= 1.3min
[CV 4/5; 1/6] START n_components=3..............................................
[CV 4/5; 1/6] END ........n_components=3;, score=-1063973.816 total time= 1.2min
[CV 5/5; 1/6] START n_components=3..............................................
[CV 5/5; 1/6] END ........n_components=3;, score=-1037903.309 total time= 1.1min
[CV 1/5; 2/6] START n_components=4..............................................
[CV 1/5; 2/6] END .........n_components=4;, score

In [56]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.061126 -0.192797       1        1  20.771344
1      0.082656  0.054320       2        1  15.868010
5      0.032371  0.032227       3        1  14.751420
2      0.106207  0.137297       4        1  13.320694
4     -0.071770  0.173637       5        1  11.320455
3      0.081420 -0.141814       6        1  10.404342
6      0.105781 -0.018529       7        1   9.008855
7     -0.397791 -0.044341       8        1   4.554881, topic_info=         Term        Freq       Total Category  logprob  loglift
1147       os  164.000000  164.000000  Default  30.0000  30.0000
1508   screen  125.000000  125.000000  Default  29.0000  29.0000
867       jre  129.000000  129.000000  Default  28.0000  28.0000
1295  project  288.000000  288.000000  Default  27.0000  27.0000
1736     test  131.000000  131.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1613    space    4.802971   16.405339   Topic8  -5.4116   1.8606
225   charact    4.489678   17.610833   Topic8  -5.4791   1.7222
1943     work    8.022798  220.650849   Topic8  -4.8986  -0.2253
259      code    7.061345  359.719637   Topic8  -5.0262  -0.8417
635      font    3.802904    9.839762   Topic8  -5.6451   2.1383

[570 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
7         1  0.994931  abstract
15        2  0.128370     activ
15        3  0.128370     activ
15        4  0.096278     activ
15        5  0.609758     activ
...     ...       ...       ...
1982      8  0.661484    xfffff
1983      8  1.043325   xffffff
1988      8  1.010460        xx
1996      5  1.087388     young
1999      3  0.882694      zoom

[1186 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 3, 5, 4, 7, 8])

In [57]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.061126 -0.192797       1        1  25.066933
5      0.032371  0.032227       2        1  16.623198
1      0.082656  0.054320       3        1  14.160305
2      0.106207  0.137297       4        1  12.687087
6      0.105781 -0.018529       5        1  11.177835
4     -0.071770  0.173637       6        1   8.977539
3      0.081420 -0.141814       7        1   6.798932
7     -0.397791 -0.044341       8        1   4.508171, topic_info=         Term        Freq       Total Category  logprob  loglift
1147       os  128.000000  128.000000  Default  30.0000  30.0000
1508   screen   98.000000   98.000000  Default  29.0000  29.0000
1295  project  222.000000  222.000000  Default  28.0000  28.0000
867       jre   97.000000   97.000000  Default  27.0000  27.0000
1736     test  118.000000  118.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
635      font    3.027379    7.127593   Topic8  -5.6451   2.2430
1613    space    3.823502   14.471334   Topic8  -5.4116   1.7683
1943     work    6.386711  180.431159   Topic8  -4.8986  -0.2419
225   charact    3.574099   15.404660   Topic8  -5.4791   1.6383
259      code    5.621327  302.358292   Topic8  -5.0262  -0.8858

[569 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
7         1  1.026463  abstract
15        2  0.138228     activ
15        3  0.138228     activ
15        4  0.092152     activ
15        6  0.552912     activ
...     ...       ...       ...
1983      8  0.873870   xffffff
1987      4  0.898788   xwindow
1988      8  0.987258        xx
1996      6  1.131722     young
1999      2  0.975593      zoom

[1137 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 2, 3, 7, 5, 4, 8])

### Вывод

Если смотреть на 8 компонент, у нас не выявилось новых фич, только лишь слились старые.  
Также нет никаких изменений в новом релизе.

Также посмотрим на обучение при небольшом количестве фич и топиков.

In [58]:
N_FEATURES = 100

In [59]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_features=N_FEATURES, stop_words="english")
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 1.013s.



In [60]:
search_params = {'n_components': [3, 4, 5, 6, 7, 8]}
model = grid_search_lda(search_params, tf)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START n_components=3..............................................
[CV 1/5; 1/6] END .........n_components=3;, score=-268419.766 total time=  51.1s
[CV 2/5; 1/6] START n_components=3..............................................
[CV 2/5; 1/6] END .........n_components=3;, score=-305958.413 total time=  58.7s
[CV 3/5; 1/6] START n_components=3..............................................
[CV 3/5; 1/6] END .........n_components=3;, score=-303326.368 total time=  44.3s
[CV 4/5; 1/6] START n_components=3..............................................
[CV 4/5; 1/6] END .........n_components=3;, score=-323157.855 total time=  46.4s
[CV 5/5; 1/6] START n_components=3..............................................
[CV 5/5; 1/6] END .........n_components=3;, score=-318142.076 total time=  45.3s
[CV 1/5; 2/6] START n_components=4..............................................
[CV 1/5; 2/6] END .........n_components=4;, score

In [61]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.245151  0.064487       1        1  14.885623
4      0.281737 -0.018646       2        1  14.810446
7     -0.139777  0.310707       3        1  14.301876
0     -0.099494 -0.002449       4        1  13.467119
5      0.058173 -0.051239       5        1  13.461022
6     -0.111343 -0.127155       6        1  10.454187
2     -0.178046 -0.094891       7        1   9.403047
1     -0.056402 -0.080813       8        1   9.216681, topic_info=         Term        Freq       Total Category  logprob  loglift
58         os  218.000000  218.000000  Default  30.0000  30.0000
66    project  278.000000  278.000000  Default  29.0000  29.0000
73     result  286.000000  286.000000  Default  28.0000  28.0000
42     instal  142.000000  142.000000  Default  27.0000  27.0000
15     consol  136.000000  136.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
25      error   13.509602  208.894199   Topic8  -4.3174  -0.3543
44  interpret   10.030520  172.635666   Topic8  -4.6151  -0.4614
55        new    8.911445  133.712660   Topic8  -4.7334  -0.3242
72   reproduc    9.017271  235.697518   Topic8  -4.7216  -0.8793
45       issu    7.961561  106.253796   Topic8  -4.8461  -0.2071

[301 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.027292  actual
0         2  0.832405  actual
0         4  0.027292  actual
0         5  0.054584  actual
0         6  0.013646  actual
...     ...       ...     ...
98        5  0.164572    work
98        6  0.151229    work
98        7  0.084510    work
98        8  0.128989    work
99        3  0.997387      xx

[497 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 8, 1, 6, 7, 3, 2])

In [62]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.245151 -0.064487       1        1  16.341623
4      0.281737  0.018646       2        1  15.394331
0     -0.099494  0.002449       3        1  14.196272
5      0.058173  0.051239       4        1  13.444463
7     -0.139777 -0.310707       5        1  12.698318
6     -0.111343  0.127155       6        1   9.989246
2     -0.178046  0.094891       7        1   9.094457
1     -0.056402  0.080813       8        1   8.841289, topic_info=         Term        Freq       Total Category  logprob  loglift
66    project  226.000000  226.000000  Default  30.0000  30.0000
58         os  151.000000  151.000000  Default  29.0000  29.0000
73     result  234.000000  234.000000  Default  28.0000  28.0000
42     instal  106.000000  106.000000  Default  27.0000  27.0000
15     consol  101.000000  101.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
44  interpret    7.477343  130.902588   Topic8  -4.6151  -0.4368
55        new    6.643119  107.675562   Topic8  -4.7334  -0.3598
49       like    5.835439   72.139719   Topic8  -4.8631  -0.0889
72   reproduc    6.722008  193.039882   Topic8  -4.7216  -0.9318
45       issu    5.935019   82.886394   Topic8  -4.8461  -0.2109

[305 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.016974  actual
0         2  0.848712  actual
0         3  0.033948  actual
0         4  0.050923  actual
0         6  0.016974  actual
...     ...       ...     ...
98        5  0.150961    work
98        6  0.145155    work
98        7  0.081287    work
98        8  0.121930    work
99        5  1.001657      xx

[483 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 6, 8, 7, 3, 2])

Выделились примерно те же топики, но лучше разделились.

Попробуем разные штуки на 15 топиках

In [63]:
N_FEATURES = 3000

print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_features=N_FEATURES, stop_words="english")
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 1.386s.



In [64]:
search_params = {'n_components': [15], 'learning_decay': [.5, .7, .9]}
model = grid_search_lda(search_params, tf)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START learning_decay=0.5, n_components=15.........................
[CV 1/5; 1/3] END learning_decay=0.5, n_components=15;, score=-1042311.638 total time= 1.4min
[CV 2/5; 1/3] START learning_decay=0.5, n_components=15.........................
[CV 2/5; 1/3] END learning_decay=0.5, n_components=15;, score=-1069364.475 total time= 1.3min
[CV 3/5; 1/3] START learning_decay=0.5, n_components=15.........................
[CV 3/5; 1/3] END learning_decay=0.5, n_components=15;, score=-1059576.818 total time= 1.4min
[CV 4/5; 1/3] START learning_decay=0.5, n_components=15.........................
[CV 4/5; 1/3] END learning_decay=0.5, n_components=15;, score=-1119650.566 total time= 1.3min
[CV 5/5; 1/3] START learning_decay=0.5, n_components=15.........................
[CV 5/5; 1/3] END learning_decay=0.5, n_components=15;, score=-1109206.110 total time= 1.4min
[CV 1/5; 2/3] START learning_decay=0.7, n_components=15..........

In [65]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.058386  0.048150       1        1  12.572246
6      0.090759  0.077132       2        1  10.048328
14     0.141142 -0.148830       3        1   9.668328
11     0.087064  0.039149       4        1   8.877473
8      0.009379  0.118078       5        1   7.700691
2      0.048938  0.054938       6        1   7.588558
13     0.087522 -0.222562       7        1   7.445553
9      0.115194 -0.089316       8        1   6.697253
7      0.060971 -0.010805       9        1   6.689991
5     -0.094420  0.075871      10        1   5.197534
12    -0.037972  0.188627      11        1   5.124553
1     -0.169808  0.183056      12        1   4.852027
0      0.031502  0.017817      13        1   4.828427
4      0.017532 -0.147845      14        1   2.404130
10    -0.446187 -0.183461      15        1   0.304908, topic_info=         Term        Freq       Total Category  logprob  loglift
1726       os  175.000000  175.000000  Default  30.0000  30.0000
1933  project  278.000000  278.000000  Default  29.0000  29.0000
2238   screen  130.000000  130.000000  Default  28.0000  28.0000
2581     test  145.000000  145.000000  Default  27.0000  27.0000
1300      jre  139.000000  139.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
635     devic    0.571044    1.213318  Topic15  -4.8592   5.0393
2225      say    1.263247   28.292791  Topic15  -4.0653   2.6840
246      bool    0.620216    4.125877  Topic15  -4.7766   3.8980
1064    handl    0.675011    9.738337  Topic15  -4.6920   3.1238
1921  process    0.482333   52.457893  Topic15  -5.0281   1.1038

[1021 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        3  1.102720  abstract
23        1  0.030298     activ
23        2  0.090895     activ
23        4  0.121193     activ
23        5  0.121193     activ
...     ...       ...       ...
2980      5  1.627082       xmx
2982      6  0.809418   xwindow
2983     11  1.005821        xx
2987     13  0.895821      yaml
2994     12  0.982182     young

[2229 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 15, 12, 9, 3, 14, 10, 8, 6, 13, 2, 1, 5, 11])

In [66]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.141142 -0.148830       1        1  13.226335
3     -0.058386  0.048150       2        1  12.506885
6     -0.090759  0.077132       3        1  11.049808
11    -0.087064  0.039149       4        1   8.739682
13    -0.087522 -0.222562       5        1   8.093052
2     -0.048938  0.054938       6        1   7.583491
7     -0.060971 -0.010805       7        1   7.349775
8     -0.009379  0.118078       8        1   6.811552
9     -0.115194 -0.089316       9        1   5.876294
12     0.037972  0.188627      10        1   5.643031
0     -0.031502  0.017817      11        1   3.958472
1      0.169808  0.183056      12        1   3.574431
5      0.094420  0.075871      13        1   2.632506
4     -0.017532 -0.147845      14        1   2.519230
10     0.446187 -0.183461      15        1   0.435458, topic_info=         Term        Freq       Total Category  logprob  loglift
1726       os  152.000000  152.000000  Default  30.0000  30.0000
2238   screen  115.000000  115.000000  Default  29.0000  29.0000
1933  project  242.000000  242.000000  Default  28.0000  28.0000
1300      jre  115.000000  115.000000  Default  27.0000  27.0000
2581     test  103.000000  103.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
635     devic    0.655178    1.147236  Topic15  -4.8592   4.8763
2225      say    1.449364   24.203166  Topic15  -4.0653   2.6212
246      bool    0.711595    3.770224  Topic15  -4.7766   3.7691
1064    handl    0.774462    8.161897  Topic15  -4.6920   3.0815
1921  process    0.553396   40.679977  Topic15  -5.0281   1.1391

[1022 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        1  1.005889  abstract
23        2  0.041781     activ
23        3  0.083561     activ
23        4  0.125342     activ
23        7  0.292464     activ
...     ...       ...       ...
2983     10  0.991225        xx
2987     11  0.886583      yaml
2994     12  1.102246     young
2998     10  0.606875      zoom
2998     13  0.606875      zoom

[2034 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 4, 7, 12, 14, 3, 8, 9, 10, 13, 1, 2, 6, 5, 11])

### Вывод

То же самое для 8-ми топиков:

In [67]:
N_FEATURES = 3000

print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_features=N_FEATURES, stop_words="english")
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 1.729s.



In [68]:
search_params = {'n_components': [8], 'learning_decay': [.5, .7, .9, 1.2]}
model = grid_search_lda(search_params, tf)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START learning_decay=0.5, n_components=8..........................
[CV 1/5; 1/4] END learning_decay=0.5, n_components=8;, score=-1035051.550 total time= 1.8min
[CV 2/5; 1/4] START learning_decay=0.5, n_components=8..........................
[CV 2/5; 1/4] END learning_decay=0.5, n_components=8;, score=-1066659.105 total time= 1.7min
[CV 3/5; 1/4] START learning_decay=0.5, n_components=8..........................
[CV 3/5; 1/4] END learning_decay=0.5, n_components=8;, score=-1057422.211 total time= 1.9min
[CV 4/5; 1/4] START learning_decay=0.5, n_components=8..........................
[CV 4/5; 1/4] END learning_decay=0.5, n_components=8;, score=-1119756.570 total time= 1.7min
[CV 5/5; 1/4] START learning_decay=0.5, n_components=8..........................
[CV 5/5; 1/4] END learning_decay=0.5, n_components=8;, score=-1097166.127 total time= 1.6min
[CV 1/5; 2/4] START learning_decay=0.7, n_components=8................

In [69]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.079594  0.085606       1        1  26.139556
2     -0.079810  0.003548       2        1  17.816619
7     -0.106742  0.039811       3        1  14.025099
0     -0.047348  0.016909       4        1  12.666462
6     -0.067574 -0.096997       5        1  11.847017
1      0.013868 -0.290789       6        1   8.060607
4     -0.076103  0.187936       7        1   5.671910
5      0.443303  0.053976       8        1   3.772731, topic_info=         Term        Freq       Total Category  logprob  loglift
1726       os  183.000000  183.000000  Default  30.0000  30.0000
2581     test  108.000000  108.000000  Default  29.0000  29.0000
2238   screen  133.000000  133.000000  Default  28.0000  28.0000
1300      jre  148.000000  148.000000  Default  27.0000  27.0000
281     build  109.000000  109.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1759    panda    6.996451   10.525992   Topic8  -4.8691   2.8689
1421     long   12.441946   27.392525   Topic8  -4.2934   2.4882
671   display    6.678502   39.394025   Topic8  -4.9156   1.5027
2125   render    5.520092   12.935410   Topic8  -5.1061   2.4258
1737   output    5.911503   46.788510   Topic8  -5.0376   1.2086

[579 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        1  0.960462  abstract
26        1  0.282930       add
26        2  0.212198       add
26        3  0.271141       add
26        4  0.082521       add
...     ...       ...       ...
2972      8  1.073940     xffff
2975      8  0.885529    xfffff
2976      8  0.928279   xffffff
2983      5  1.005628        xx
2994      6  1.025612     young

[1196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 8, 1, 7, 2, 5, 6])

In [70]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.079594  0.085606       1        1  30.232048
2     -0.079810  0.003548       2        1  15.914771
7     -0.106742  0.039811       3        1  14.179561
6     -0.067574 -0.096997       4        1  12.702366
0     -0.047348  0.016909       5        1  12.573119
4     -0.076103  0.187936       6        1   6.877004
1      0.013868 -0.290789       7        1   4.915182
5      0.443303  0.053976       8        1   2.605949, topic_info=         Term        Freq       Total Category  logprob  loglift
2581     test  103.000000  103.000000  Default  30.0000  30.0000
1726       os  152.000000  152.000000  Default  29.0000  29.0000
2238   screen  114.000000  114.000000  Default  28.0000  28.0000
1300      jre  115.000000  115.000000  Default  27.0000  27.0000
1214   instal  104.000000  104.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1421     long    6.904148   19.599014   Topic8  -4.2934   2.6040
1759    panda    3.882394    6.844187   Topic8  -4.8691   3.0804
671   display    3.705961   30.424695   Topic8  -4.9156   1.5421
2125   render    3.063149    9.091864   Topic8  -5.1061   2.5594
1737   output    3.280347   34.033205   Topic8  -5.0376   1.3080

[575 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
10        1  1.036536  abstract
26        1  0.312053       add
26        2  0.184395       add
26        3  0.269500       add
26        4  0.070921       add
...     ...       ...       ...
2975      8  0.794199    xfffff
2976      8  0.834384   xffffff
2983      4  0.993272        xx
2987      5  0.818190      yaml
2994      7  1.041511     young

[1131 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 8, 7, 1, 5, 2, 6])

Темы повторяются, новых не появилось, различий опять же нет.

25 топиков

In [71]:
N_FEATURES = 1000

print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_features=N_FEATURES, stop_words="english")
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 0.913s.



In [72]:
search_params = {'n_components': [25], 'learning_decay': [.5, .7, .9]}
model = grid_search_lda(search_params, tf)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START learning_decay=0.5, n_components=25.........................
[CV 1/5; 1/3] END learning_decay=0.5, n_components=25;, score=-843593.380 total time= 1.2min
[CV 2/5; 1/3] START learning_decay=0.5, n_components=25.........................
[CV 2/5; 1/3] END learning_decay=0.5, n_components=25;, score=-884779.110 total time= 1.2min
[CV 3/5; 1/3] START learning_decay=0.5, n_components=25.........................
[CV 3/5; 1/3] END learning_decay=0.5, n_components=25;, score=-876632.204 total time= 1.2min
[CV 4/5; 1/3] START learning_decay=0.5, n_components=25.........................
[CV 4/5; 1/3] END learning_decay=0.5, n_components=25;, score=-924098.549 total time= 1.2min
[CV 5/5; 1/3] START learning_decay=0.5, n_components=25.........................
[CV 5/5; 1/3] END learning_decay=0.5, n_components=25;, score=-912464.610 total time= 1.2min
[CV 1/5; 2/3] START learning_decay=0.7, n_components=25...............

In [73]:
visualise_topics(data_v2020_2_str, model, tf_vectorizer, "v2020_2.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.031070 -0.053511       1        1  10.041939
0     -0.092489 -0.067502       2        1   7.864103
10    -0.006952 -0.094147       3        1   5.588140
13     0.066378 -0.132103       4        1   5.341948
1      0.251232 -0.103339       5        1   5.272632
2     -0.139309  0.007407       6        1   5.175568
20     0.188625 -0.143003       7        1   5.039531
17    -0.113358 -0.007875       8        1   4.866665
12    -0.023221  0.215873       9        1   4.782689
22     0.094674  0.213108      10        1   4.576211
14    -0.089999 -0.114626      11        1   4.286901
7     -0.101107  0.107550      12        1   4.257871
3     -0.046307 -0.152447      13        1   4.198181
23    -0.118375  0.067105      14        1   3.584920
4     -0.098721 -0.025177      15        1   3.427399
5     -0.035746 -0.094281      16        1   3.314323
16    -0.119692  0.044249      17        1   3.217354
24     0.295439 -0.021437      18        1   3.083645
6     -0.027255 -0.089110      19        1   2.958633
8     -0.152449 -0.070535      20        1   2.900570
18     0.046568 -0.102474      21        1   2.632742
15    -0.117403 -0.015369      22        1   1.679341
21     0.278689  0.084845      23        1   0.895785
19    -0.038697  0.211021      24        1   0.802418
11     0.068403  0.335780      25        1   0.210491, topic_info=        Term        Freq       Total Category  logprob  loglift
274    error  357.000000  357.000000  Default  30.0000  30.0000
599       os  188.000000  188.000000  Default  29.0000  29.0000
885     test  125.000000  125.000000  Default  28.0000  28.0000
670  project  294.000000  294.000000  Default  27.0000  27.0000
775   screen  136.000000  136.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
663  process    0.407028   52.686470  Topic25  -4.7379   1.3003
348       gc    0.175229   12.534534  Topic25  -5.5807   1.8933
182    crash    0.201807   19.565740  Topic25  -5.4395   1.5893
971       vm    0.243303   76.450134  Topic25  -5.2525   0.4134
432      int    0.177301   22.227191  Topic25  -5.5689   1.3323

[1167 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3        11  0.769936   absolut
4        21  1.017448  abstract
5         1  0.120005    accept
5         5  0.480021    accept
5        14  0.120005    accept
...     ...       ...       ...
995       1  0.251578       xml
995      11  0.503156       xml
995      15  0.251578       xml
996      12  1.011320        xx
997      11  0.953172      yaml

[3391 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 11, 14, 2, 3, 21, 18, 13, 23, 15, 8, 4, 24, 5, 6, 17, 25, 7, 9, 19, 16, 22, 20, 12])

In [74]:
visualise_topics(data_v2020_3_str, model, tf_vectorizer, "v2020_3.html")

/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/Caskroom/miniconda/base/envs/hse/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.031070  0.053511       1        1  10.467971
13    -0.066378  0.132103       2        1   7.004363
0      0.092489  0.067502       3        1   6.614826
1     -0.251232  0.103339       4        1   6.277765
20    -0.188625  0.143003       5        1   6.188462
3      0.046307  0.152447       6        1   5.155801
14     0.089999  0.114626       7        1   5.049287
17     0.113358  0.007875       8        1   4.999368
7      0.101107 -0.107550       9        1   4.249836
10     0.006952  0.094147      10        1   4.247981
12     0.023221 -0.215873      11        1   3.834825
2      0.139309 -0.007407      12        1   3.822193
24    -0.295439  0.021437      13        1   3.738184
16     0.119692 -0.044249      14        1   3.674868
8      0.152449  0.070535      15        1   3.629226
4      0.098721  0.025177      16        1   3.432019
6      0.027255  0.089110      17        1   3.322947
5      0.035746  0.094281      18        1   3.172243
18    -0.046568  0.102474      19        1   3.047619
23     0.118375 -0.067105      20        1   2.513495
15     0.117403  0.015369      21        1   1.691433
22    -0.094674 -0.213108      22        1   1.477885
21    -0.278689 -0.084845      23        1   1.050328
19     0.038697 -0.211021      24        1   0.964361
11    -0.068403 -0.335780      25        1   0.372716, topic_info=         Term        Freq       Total Category  logprob  loglift
599        os  143.000000  143.000000  Default  30.0000  30.0000
274     error  126.000000  126.000000  Default  29.0000  29.0000
885      test  112.000000  112.000000  Default  28.0000  28.0000
670   project  242.000000  242.000000  Default  27.0000  27.0000
775    screen  108.000000  108.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
25   anaconda    0.247463    8.655291  Topic25  -5.5817   2.0374
663   process    0.575385   34.871330  Topic25  -4.7379   1.4877
182     crash    0.285279   12.612113  Topic25  -5.4395   1.8032
971        vm    0.343939   44.175529  Topic25  -5.2525   0.7366
432       int    0.250637   21.206487  Topic25  -5.5689   1.1541

[1161 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3         7  0.826163   absolut
4        19  1.102197  abstract
5         1  0.138775    accept
5         4  0.555100    accept
5        20  0.138775    accept
...     ...       ...       ...
994      25  0.950432   xffffff
995       1  0.288850       xml
995       7  0.577699       xml
996       9  0.990577        xx
997       7  1.022584      yaml

[3075 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 14, 1, 2, 21, 4, 15, 18, 8, 11, 13, 3, 25, 17, 9, 5, 7, 6, 19, 24, 16, 23, 22, 20, 12])

## Глобальный вывод

В итоге выделяется около 8 нормальных тем, которые сохраняются в новом релизе и новых не добавляется.